# **Machine Learning from Data**

## Lab 3: Feature Selection

2024 - Veronica Vilaplana - [GPI @ IDEAI](https://imatge.upc.edu/web/) Research group

Based on
* Scikit-Learn documentation and examples
* [A short guide for feature engineering and feature selection](https://github.com/Yimeng-Zhang/feature-engineering-and-feature-selection/blob/master/A%20Short%20Guide%20for%20Feature%20Engineering%20and%20Feature%20Selection.pdf), by Yimeng-Zhang
* [A feature selection tutorial](https://github.com/slowvak/FeatureSelectionTutorial/blob/main/FeatureSelection.ipynb)


#Feature Selection

Feature selection is the process of identifying and selecting a subset of relevant features for use in machine learning model building. Contrary to the belief that "more data is always better," including irrelevant or redundant features can actually hinder model performance.

By employing feature selection, we can:

* Simplify models: Leading to improved interpretability.
* Reduce training time and computational cost.
* Lower data collection costs.
* Mitigate the curse of dimensionality.
* Enhance generalization by reducing overfitting.

It's important to note that the optimal feature subset can vary depending on the specific machine learning algorithm. Therefore, feature selection should be considered an integral part of the model building process, rather than a separate step. For example, when selecting features for a linear model, techniques like regression coefficient importance or Lasso regularization are particularly effective. For tree-based models, tree-derived feature importance can be a valuable tool.


Feature selectio methods can be broadly classified into **unsupervised** or **supervised methods**


**Unsupervised methods** are such methods that do not make use of any labels. In other words, they don’t need access to the target variable of the machine learning model.
We might want to discard the features with:
* Zero or near-zero variance. Features that are (almost) constant provide little information to learn from and thus are irrelevant.
* Many missing values. While dropping incomplete features is not the preferred way to handle missing data, it is often a good start, and if too many entries are missing, it might be the only sensible thing to do since such features are likely inconsequential.
* High multicollinearity; multicollinearity means a strong correlation between different features, which might signal redundancy issues. This method eliminates highly correlated features to reduce redundancy.

In turn, **supervised methods** can be grouped into  **filter methods**,  **wrapper methods** and **embedded methods**



**Filter methods:** select features based on a performance measure regardless of the ML algorithm later employed.  Univariate filters evaluate and rank a single feature according to a certain criteria, while multivariate filters evaluate the entire feature space. As a result, filter methods are suited for a first step quick screen and removal of irrelevant features.

Most common filter methods are:
* Univariate statistical tests: Compute chi-squared tests for categorial features and ANOVA for numerical features
* Mutual information filter: Mutual information measures how much information the presence/absence of a feature contributes to making the correct prediction of the target variable.

**Wrapper methods:**  use a search strategy to search through the space of possible feature subsets and evaluate each subset by the quality of the performance on a ML algorithm. Practically any combination of search strategy
and algorithm can be used as a wrapper.

The most common search strategy group is Sequential search, including Forward Selection and Backward Elimination, or Recursive Feature Elimination.

Another key element in wrappers is stopping criteria. When to stop the search? In general there're three: performance increase, performance decrease or
predefined number of features is reached.

For example, *Forward Feature Selection* starts with an empty set and iteratively adds the most informative feature. It starts by evaluating all features individually and selects the one that generates the best performing algorithm, according to a pre-set evaluation criteria. In the second step, it evaluates all possible combinations of the selected feature and a second feature, and selects the pair that produce the best performing algorithm based on the same pre-set criteria. The pre-set criteria can be the roc_auc for classification and the r squared for regression for example.
This selection procedure evaluates all possible single, double, triple and so on feature combinations. Therefore, it is quite computationally expensive, and sometimes, if feature space is big, even unfeasible.

*Backward Feature Elimination* starts with the full set of features and iteratively removes the least informative feature.

*Recursive Feature Elimination (RFE)* recursively removes features based on their importance scores from a model.

**Embedded Methods:** combine the advantages of the filter and wrapper methods. A learning algorithm takes advantage of its own variable selection process and performs feature selection and classification at same
time. Common embedded methods include

*Lasso*, that penalizes the model's coefficients, effectively shrinking less important features towards zero and various types of

*Tree-based algorithms*, where feature importance scores can be derived from the tree structure.

They are less computationally expensive as they only train the model once, compared to Wrappers.


The following are the feature selection strategies implemented in Scikit-Learn.

There are other libraries with more methods, for example
* [Py-FS](https://pypi.org/project/Py-FS/)
* [Feature engine](https://feature-engine.trainindata.com/en/latest/)
* [Feature engineering and feature selection](https://github.com/Yimeng-Zhang/feature-engineering-and-feature-selection/blob/master/4.1_Demo_Feature_Selection_Filter.ipynb)

## 1. Removing features with low variance

`VarianceThreshold` is a simple baseline approach to feature selection. It removes all features whose variance doesn’t meet some threshold. By default, it removes all zero-variance features, i.e. features that have the same value in all samples.

As an example, suppose that we have a dataset with boolean features, and we want to remove all features that are either one or zero (on or off) in more than 80% of the samples. Boolean features are Bernoulli random variables, and the variance of such variables is given by

$Var[X]= p(1-p)$

so we can select using the threshold .8 * (1 - .8):

In [1]:
from sklearn.feature_selection import VarianceThreshold
X = [[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]]
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(X)

array([[0, 1],
       [1, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 1]])

## 2. Univariate feature selection

Univariate feature selection works by selecting the best features based on univariate statistical tests. It can be seen as a preprocessing step to an estimator. Scikit-learn exposes feature selection routines as objects that implement the transform method:

* `SelectKBest` removes all but the
 highest scoring features

* `SelectPercentile` removes all but a user-specified highest scoring percentage of features using common univariate statistical tests for each feature: false positive rate SelectFpr, false discovery rate SelectFdr, or family wise error SelectFwe.

* `GenericUnivariateSelect` allows to perform univariate feature selection with a configurable strategy. This allows to select the best univariate selection strategy with hyper-parameter search estimator.

For instance, we can use a F-test to retrieve the two best features for a dataset as follows:

In [2]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
X, y = load_iris(return_X_y=True)
print(X.shape)
X_new = SelectKBest(f_classif, k=2).fit_transform(X, y)
print(X_new.shape)

(150, 4)
(150, 2)


These objects take as input a scoring function that returns univariate scores and p-values (or only scores for SelectKBest and SelectPercentile):

* For regression: r_regression, f_regression, mutual_info_regression

* For classification: chi2, f_classif, mutual_info_classif

The methods based on F-test estimate the degree of linear dependency between two random variables.


This [notebook](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_feature_selection.html#sphx-glr-auto-examples-feature-selection-plot-feature-selection-py) is an example of using univariate feature selection to improve classification accuracy on a noisy dataset (using Support Vector Machines).

## 3. Recursive feature elimination

Given an external estimator that assigns weights to features (e.g., the coefficients of a linear model), the goal of recursive feature elimination (RFE) is to select features by recursively considering smaller and smaller sets of features. First, the estimator is trained on the initial set of features and the importance of each feature is obtained either through any specific attribute (such as coef_, feature_importances_) or callable. Then, the least important features are pruned from current set of features. That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.

See for example [Recursive feature elimination](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_digits.html#sphx-glr-auto-examples-feature-selection-plot-rfe-digits-py): A recursive feature elimination example showing the relevance of pixels in a digit classification task.

## 4. Feature selection using SelectFromModel

`SelectFromModel` is a meta-transformer that can be used alongside any estimator that assigns importance to each feature through a specific attribute. The features are considered unimportant and removed if the corresponding importance of the feature values are below the provided threshold parameter.

For example, we can use the **L1-based Feature selection**

Linear models penalized with the L1 norm have sparse solutions: many of their estimated coefficients are zero. When the goal is to reduce the dimensionality of the data to use with another classifier, they can be used along with `SelectFromModel` to select the non-zero coefficients. In particular, sparse estimators useful for this purpose are the `Lasso` for regression, and of `LogisticRegression` and `LinearSVC` for classification:


In [3]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
X, y = load_iris(return_X_y=True)
print(X.shape)
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
print(X_new.shape)

(150, 4)
(150, 3)


With SVMs and logistic-regression, the parameter C controls the sparsity: the smaller C the fewer features selected. With Lasso, the higher the alpha parameter, the fewer features selected.


See more information on Scikit-Learn feature selection methods [here](https://scikit-learn.org/stable/modules/feature_selection.html).

## 5. Examples. Feature selection on the Pima Indians Diabetes Dataset

We will try some feature selection methods using the Pima Indians Diabetes Dataset

In [4]:
import pandas as pd
import numpy as np

First, we need to get the data. Upload the file 'diabetes.csv'

In [5]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv("drive/MyDrive/MLEARN/Lab/Lab3/diabetes.csv")
# print out the first few lines to make sure we got the data and that it looks reasonable
data.head()

ValueError: mount failed

Next we import modules for visualizing data and making plots. Here will do a quick plot to see the correlation coefficients of the various features in the data.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

#Lets look at how much the various features are correlated using Pearson Correlation
plt.figure(figsize=(12,10))
cor = data.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

Next we convert the dataframe into a set of feature vectors 'X' and the class or label which we will call 'y', which in our case is 1-diabetic 0-not diabetic.

In [ ]:
array = data.values
X = array[:,0:8]
y = array[:,8]

#print out the shape of the X feature array--number of features and number of rows/examples
X.shape

We will train a linear classifier (LDA), and compute baseline accuracy and error with all the features.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=109) # 70% training and 30% test

lda = LinearDiscriminantAnalysis(solver="svd",store_covariance=True)

ldamodel = lda.fit(X_train, y_train)
y_tpred_lda = ldamodel.predict(X_train)
y_testpred_lda = ldamodel.predict(X_test)

lda_train_accuracy = accuracy_score(y_train,y_tpred_lda)
lda_train_error = 1. - lda_train_accuracy
print('LDA train accuracy: %f' %lda_train_accuracy)
print('LDA train error: %f' %lda_train_error)
print('LDA train confusion matrix:')
print(confusion_matrix(y_train,y_tpred_lda))


lda_test_accuracy = accuracy_score(y_test,y_testpred_lda)
lda_test_error = 1. - lda_train_accuracy
print('LDA test accuracy: %f' %lda_test_accuracy)
print('LDA test error: %f' %lda_test_error)
print('LDA test confusion matrix:')
print(confusion_matrix(y_test,y_testpred_lda))


Now we will use some feature selection methods.
We start with filter method SelectKBest, based on a F-test that estimates the degree of linear dependency between two random variables (feature anf target).

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=109) # 70% training and 30% test

# Feature extraction--select the 4 best features
test = SelectKBest(score_func=f_classif, k=5)
fit = test.fit(X_train, y_train)

# Summarize scores
np.set_printoptions(precision=3)
feature_names = data.columns[0:8]

#print (feature_names) and (fit.scores_)
for i, feature in enumerate (feature_names):
    print (f'{fit.scores_[i]}, {feature}')


Let's sort the list of values


In [ ]:
zipped_lists = zip(fit.scores_, feature_names)
sorted_pairs = sorted(zipped_lists)
for f, val in sorted_pairs:
    print (f'{f} - {val}')

Now, we will train the linear classifier using only the last five features. We first remove the features from the whole set and then split it again into train an test subsets

In [ ]:
#First, need to remove the 'Outcome' column since that is the target
fclas_df = data.drop(columns=["Outcome"])

# delete the columns we don't want
fclas_df = fclas_df.drop(columns=["BloodPressure","SkinThickness","Insulin"])

d_fclas = fclas_df.values
X_1 = d_fclas[:,:]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y, test_size=0.3, random_state=109) # 70% training and 30% test

lda = LinearDiscriminantAnalysis(solver="svd",store_covariance=True)

ldamodel = lda.fit(X_train, y_train)
y_tpred_lda = ldamodel.predict(X_train)
y_testpred_lda = ldamodel.predict(X_test)


lda_train_accuracy = accuracy_score(y_train,y_tpred_lda)
lda_train_error = 1. - lda_train_accuracy
print('LDA train accuracy: %f' %lda_train_accuracy)
print('LDA train error: %f' %lda_train_error)
print('LDA train confusion matrix:')
print(confusion_matrix(y_train,y_tpred_lda))


lda_test_accuracy = accuracy_score(y_test,y_testpred_lda)
lda_test_error = 1. - lda_train_accuracy
print('LDA test accuracy: %f' %lda_test_accuracy)
print('LDA test error: %f' %lda_test_error)
print('LDA test confusion matrix:')
print(confusion_matrix(y_test,y_testpred_lda))


print (f'Accuracy on the test set with the best 5 features using F-test is {lda_test_accuracy}')


### <font color='red'>TO_DO: try adding or removing features and see the impact on performance</font>



### Next we will apply a wrapper method--Recursive Feature Elimination or RFE



In [ ]:
# Split original data (with all features)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=109) # 70% training and 30% test

# Import your necessary dependencies
from sklearn.feature_selection import RFE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


estimator = LinearDiscriminantAnalysis(solver="svd",store_covariance=True)

# we will ask for the top 5 features
selector = RFE(estimator, n_features_to_select=5, step=1)
selector = selector.fit(X_train, y_train)
print(selector.support_)
rank=selector.ranking_
print ("Or if you want to see the ranking of those that didn't make the cut...")
print(rank)

We remove the features and train the classifier

In [ ]:
# drop all columns not '1'
X_1 = data.drop(data.columns[8], axis=1)  # get rid of 'class'
for i in range(7):
    if selector.support_[i] == False:
        X_1 = X_1.drop(X_1.columns[i], axis=1)

In [ ]:
# split the dataset X1 (using only 4 features)
X_train, X_test, y_train, y_test = train_test_split(X_1, y, test_size=0.3, random_state=109) # 70% training and 30% test

lda = LinearDiscriminantAnalysis(solver="svd",store_covariance=True)

ldamodel = lda.fit(X_train, y_train)
y_tpred_lda = ldamodel.predict(X_train)
y_testpred_lda = ldamodel.predict(X_test)


lda_train_accuracy = accuracy_score(y_train,y_tpred_lda)
lda_train_error = 1. - lda_train_accuracy
print('LDA train accuracy: %f' %lda_train_accuracy)
print('LDA train error: %f' %lda_train_error)
print('LDA train confusion matrix:')
print(confusion_matrix(y_train,y_tpred_lda))


lda_test_accuracy = accuracy_score(y_test,y_testpred_lda)
lda_test_error = 1. - lda_train_accuracy
print('LDA test accuracy: %f' %lda_test_accuracy)
print('LDA test error: %f' %lda_test_error)
print('LDA test confusion matrix:')
print(confusion_matrix(y_test,y_testpred_lda))


print (f'Test accuracy with best 5 features using RFE is {lda_test_accuracy}')


Finally we will use the same method, varying the number of selected features:

In [ ]:
# split the original dataset with all features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=109) # 70% training and 30% test

# nof - number of features that we have
nof=8
nof_list=np.arange(1,nof+1)
high_score=0
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

score_list =[]
best_nof = 0
for n in range(nof-1):
    model = LinearDiscriminantAnalysis(solver="svd",store_covariance=True)

    rfe = RFE(model,n_features_to_select=nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)

    score_list.append(score)
    print(f" case: {n}, best {best_nof} features, scoring {high_score}")
    if(score>high_score):
        high_score = score
        best_nof = nof_list[n]

print(f"  Optimal case: {best_nof} features, scoring {high_score}")